# tootltips

In [8]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

{"count":0,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    71  100    71    0     0   1543      0 --:--:-- --:--:-- --:--:--  1543


# Imports

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,parallel_bulk
import time

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [2]:
def clean_desc_txt(x):
    bad_chars_to_space = ['"', "'", "''", '""',  # these char in addr field generate errors on addok
                      ",",  # removed by precaution since we dump comma sep csv and we dont want quotechar
                      "\\n", "\\t","\\r","\\r\\n"  #
                      "\n", "\t",  # remove carriage return/tab
                      "/", "\\", "(", ")"  # remove special separators characters and ()
                                      "[", "]", "_", "°", "^","<br>", ">", "<"
                                                                    '«', '»',

                      ]
    for bad_char in bad_chars_to_space:
        x=x.replace(bad_char,' ')
    return x

In [3]:
def setup_es_client(index_name):
    es_client = Elasticsearch()

    index_name = index_name
    index_configurations = {
        "settings": {
            "index": {
                "number_of_shards": 4,
                "number_of_replicas": 0,
                "max_result_window": 5000000,

            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "text_to_analyze": {
                    "type": "text",
                    #"analyzer": "french"      
                },
                "category_index":{
                    'type':'keyword'
                }
            }
        }
    }

    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass
    es_client.indices.create(
        index=index_name,
        body=index_configurations
    )
    return es_client

# fonctions

In [4]:
import uuid

In [227]:
from generate_dpe_annexes_scripts.utils import strip_accents

sleepers = {'after_bulk':1,
           'after_search':0.2}
def gendata(index_name,id_data,data):
    for id_,vr in zip(id_data,data):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr
        }

def generate_instruction_from_list(char_list):
    
    new_char_list=list()
    for char in char_list:
        if isinstance(char,str):
            new_char_list.append(char)
        elif isinstance(char,tuple):
            new_char_list.append('('+') OR ('.join(char)+')')
            
    search_instruction = '('+') AND ('.join(new_char_list) +')'
    search_instruction = search_instruction.replace('AND (NOT) AND','AND NOT')
    search_instruction = search_instruction.replace('(NOT) AND','NOT')

    return search_instruction

def search_from_search_dict(es_client,search_dict,index_name):
    s_list = list()
    for label,char_list in  search_dict.items():
        time.sleep(sleepers['after_search'])
        search_instruction =generate_instruction_from_list(char_list)
        search_body = {
            "query": {
                "query_string": {
                    "query": search_instruction,
                    "default_field": "text_to_analyze"
                },

            },

        }
        #es_client.count(index=index_name, body=search_body)
        a_dict=es_client.search(index=index_name, body=search_body,size=5000000)

        hits = a_dict['hits']['hits']

        s=pd.Series(index=[el['_source']['id'] for el in hits],dtype='str')
        s[:]=label
        s_list.append(s)
    s_all=pd.concat(s_list)
    return s_all

def search_and_affect(data_to_search,id_col,val_col,search_dict):
    index_name = uuid.uuid4()
    es_client = setup_es_client(index_name)
    bulk(es_client, gendata(index_name,data_to_search[id_col],data_to_search[val_col]))
    time.sleep(sleepers['after_bulk'])

    res_serie = search_from_search_dict(es_client,search_dict,index_name=index_name)

    res_table=res_serie.to_frame('label')

    res_table.index.name = 'id'
    res_table=res_table.reset_index()
    # destroy index a la fin
    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass

# #     grp = df_drop.groupby('id').label.apply(lambda x: ' + '.join(sorted(list(set(x))))).reset_index()
# #     m=data.merge(grp,on='id',how='left')

#     m.label=m.label.fillna('indetermine')
    return res_table
def categorize_search_res(table,label_cat=None,category_dict=None):
    
    if category_dict is not None:
        table['category']=table.label.replace(category_dict)
    
    if label_cat is not None:
        table['label']=pd.Categorical(table.label,categories=label_cat,ordered=True)

    #df_drop = df.sort_values('label').drop_duplicates(subset=['id','category'])
    
    table.label = table.label.fillna('indetermine')
    return table

def agg_concat_label_text(table,col_label='label',col_vr='valeur_renseignee'):
    table[[col_label,col_vr]]=table[[col_label,col_vr]].fillna('indetermine')
    agg = table.groupby('td001_dpe_id').agg({

    col_label:lambda x: ' + '.join(sorted(list(set(x)))),
    col_vr:lambda x: ' + '.join(sorted(list(set(x))))
    })

    for col in agg:
        agg[col] =  agg[col].apply(lambda x:' + '.join(sorted(list(set([el for el in x.split(' + ') if el!='indetermine'])))))
        agg[col] = agg[col].replace('','indetermine')
    return agg

# load tables

In [6]:
from generate_dpe_annexes_scripts.utils import reload_package

In [7]:
import generate_dpe_annexes_scripts.trtvtables
reload_package(generate_dpe_annexes_scripts.trtvtables)

In [8]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths
from generate_dpe_annexes_scripts.trtvtables import DPETrTvTables

trtv = DPETrTvTables()

if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        if dept_dir.name=='38':
            print(dept_dir)
            annexe_dept_dir = annexe_dir / dept_dir.name
            annexe_dept_dir.mkdir(exist_ok=True, parents=True)
            # LOAD TABLES
            td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
            td002 = pd.read_csv(dept_dir/'td002_consommation.csv',dtype=str)
            td016 = pd.read_csv(dept_dir/'td016_facture.csv',dtype=str)

            td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
            td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
            td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
            td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
            td011_p = pd.read_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv', dtype=str)

            td012_p = pd.read_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv', dtype=str)
            td014_p = pd.read_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv', dtype=str)
            td001_sys_ch= pd.read_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv', dtype=str)
            td001_sys_ecs= pd.read_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv', dtype=str)

            gen= pd.read_csv(annexe_dept_dir / 'td001_annexe_generale.csv', dtype=str)
            td001_sys_ch.columns = [col.replace('_chauffage','_ch') for col in td001_sys_ch]

            print(dept_dir)
            
            # ENVELOPPE PROCESSING
            td001 =td001.rename(columns={'id':'td001_dpe_id'})
            td011_p['type_installation_ch']=td011_p.tv025_type_installation.replace({'Chauffage Individuel':'individuel',
                                        'Chauffage Collectif':'collectif'})
            break

tv025_intermittence_type_ch not found
tv026_classe_inertie_plancher_bas not found
tv026_classe_inertie_plancher_haut not found
tv026_classe_inertie_paroi_verticale not found
tv026_classe_inertie_classe_inertie not found
tv025_intermittence_type_emetteur not found
tv029_simu_type_distribution not found
D:\data\dpe_full\depts\38
D:\data\dpe_full\depts\38


In [9]:
td014_p.is_ecs_solaire = td014_p.is_ecs_solaire.replace({'False':False,'True':True})

# dump new

In [10]:
import main
import generate_dpe_annexes_scripts.conversion_normalisation
import generate_dpe_annexes_scripts.td011_td012_processing
import generate_dpe_annexes_scripts.td013_td014_processing
import generate_dpe_annexes_scripts.td001_merge
import generate_dpe_annexes_scripts.text_matching_dict

In [11]:
reload_package(main)
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)

reload_package(generate_dpe_annexes_scripts.td011_td012_processing)
reload_package(generate_dpe_annexes_scripts.td013_td014_processing)
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
reload_package(generate_dpe_annexes_scripts.td001_merge)
reload_package(main)
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)

reload_package(generate_dpe_annexes_scripts.td011_td012_processing)
reload_package(generate_dpe_annexes_scripts.td013_td014_processing)
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
reload_package(generate_dpe_annexes_scripts.td001_merge)

In [12]:
from main import *

In [13]:
td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
td013 = pd.read_csv(dept_dir / 'td013_installation_ecs.csv', dtype=str)
td014 = pd.read_csv(dept_dir / 'td014_generateur_ecs.csv', dtype=str)

td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg = run_system_processing(td001, td006,
                                                                                                td011, td012,
                                                                                                td013, td014)
round_float_cols(td001_sys_ch_agg).to_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv')
round_float_cols(td001_sys_ecs_agg).to_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv')
round_float_cols(td011_p).to_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv')
round_float_cols(td012_p).to_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv')
round_float_cols(td013_p).to_csv(annexe_dept_dir / 'td013_installation_ecs_annexe.csv')
round_float_cols(td014_p).to_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv')

In [14]:
td001_sys_ch.type_installation_ch_concat=td001_sys_ch.type_installation_ch_concat.str.replace('Chauffage Individuel',
                                                                                              'individuel').str.replace('Chauffage Collectif',
                                                                                                                        'collectif')
td001 =td001.rename(columns={'id':'td001_dpe_id'})

# td002/td016 processing

In [15]:
from generate_dpe_annexes_scripts.td002_td016_processing import merge_td002_td016_trtrv,extract_type_energie_from_td002_td016

In [16]:
td002_p,td016_p=merge_td002_td016_trtrv(td002,td016)
td001_ener_from_facture = extract_type_energie_from_td002_td016(td001,td002_p,td016_p)

In [17]:
td001_ener_from_facture

,td001_dpe_id,td002_type_energie_ch,td002_type_energie_ecs,td016_type_energie_ch,td016_type_energie_ecs
0,291,electricite,electricite,NaN,NaN
1,423,gaz,electricite,NaN,NaN
2,435,autre,autre,NaN,NaN
3,493,bois + electricite,electricite,NaN,NaN
4,708,autre,electricite,NaN,NaN
...,...,...,...,...,...
170231,9431119,gaz,gaz,NaN,NaN
170232,9431213,NaN,NaN,NaN,NaN
170233,9431307,gaz,electricite,NaN,NaN
170234,9431405,gaz,gaz,NaN,NaN


# load dicts

In [220]:
from generate_dpe_annexes_scripts.utils import reload_package

In [301]:
import generate_dpe_annexes_scripts.text_matching_dict 
import os
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
from generate_dpe_annexes_scripts.text_matching_dict import *

from generate_dpe_annexes_scripts.conversion_normalisation import *
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)
from generate_dpe_annexes_scripts.conversion_normalisation import *


In [302]:
gen_ch_search_dict_flat.keys() -set(td012_gen_ch_search_dict.keys())

{'chaudiere bois exact',
 'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'pac air/eau en releve de chaudiere',
 'pac eau/eau en releve de chaudiere',
 'pac geothermique en releve de chaudiere',
 'pac indetermine',
 'panneaux rayonnants electriques',
 'poele ou insert indetermine',
 'radiateurs gaz'}

In [223]:
set(td014_gen_ecs_search_dict.keys())-gen_ecs_search_dict_flat.keys()

{'chaudiere bois', 'ecs thermodynamique electrique(pompe a chaleur ou ballon)'}

In [224]:
gen_ecs_search_dict_flat.keys() -set(td014_gen_ecs_search_dict.keys())

{'abscence ecs solaire',
 'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'chauffe-eau independant indetermine',
 'ecs solaire',
 'ecs thermodynamique electrique(pac ou ballon)',
 'production mixte fioul',
 'production mixte gaz',
 'production mixte indetermine'}

# search ECS

In [229]:

td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.fillna('indetermine')

td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.apply(lambda x:clean_desc_txt(x))



In [230]:


m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_search_dict_flat)
m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')


m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ecs_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')



m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict)

energie_ecs_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')


In [231]:
td003_ecs = td003.loc[td003.tr007_type_descriptif_id == '10']
td003_ecs.descriptif = td003_ecs.descriptif.fillna('indetermine')
td003_ecs.descriptif = td003_ecs.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ecs.descriptif = td003_ecs.descriptif.apply(lambda x:clean_desc_txt(x))



In [232]:

m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=gen_ecs_search_dict_flat)
m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_desc=m.merge(td003_ecs[['id','td001_dpe_id']],how='left')

m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=installation_dict)

type_installation_ecs_desc=m.merge(td003_ecs[['id','td001_dpe_id']],how='left')



m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=energie_dict)

energie_ecs_desc=m.merge(td003_ecs[['id','td001_dpe_id']],how='left')


# search Chauffage

In [303]:
from generate_dpe_annexes_scripts import text_matching_dict
reload_package(text_matching_dict)
from generate_dpe_annexes_scripts.text_matching_dict import solaire_ch_search_dict

In [304]:

td005_ch = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '16']
td005_ch.descriptif = td005_ch.valeur_renseignee.fillna('indetermine')

td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.apply(lambda x:clean_desc_txt(x))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [235]:

m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ch_search_dict_flat)

m = categorize_search_res(m,label_cat = list(gen_ch_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ch)

gen_ch_lib_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')


m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')



m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict)

energie_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')

m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=solaire_ch_search_dict)
solaire_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')


In [236]:
td003_ch = td003.loc[td003.tr007_type_descriptif_id == '11']
td003_ch.descriptif = td003_ch.descriptif.fillna('indetermine')
td003_ch.descriptif = td003_ch.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ch.descriptif = td003_ch.descriptif.apply(lambda x:clean_desc_txt(x))
  

In [237]:

m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=gen_ch_search_dict_flat)

m = categorize_search_res(m,label_cat = list(gen_ch_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ch)
gen_ch_lib_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')


m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=installation_dict)

type_installation_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')



m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=energie_dict)

energie_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')


m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=solaire_ch_search_dict)

solaire_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')


# fusion

## méthodologie de fusion

TBD

In [396]:
td001_sys=td001.rename(columns ={'id':'td001_dpe_id'})[['td001_dpe_id','tr002_type_batiment_id']].merge(td001_sys_ch[['td001_dpe_id',
                                                                                             'gen_ch_lib_infer_concat',
                                                                                            ]],on='td001_dpe_id',
                                                                              how='left')
td001_sys=td001_sys.merge(td001_sys_ecs[['td001_dpe_id','gen_ecs_lib_infer_concat']],on='td001_dpe_id',
                                                                              how='left')

## Chauffage


### préparation de la donnée

#### generateur chauffage

In [397]:
# libéllés des generateurs chauffage issues des données textes
gen_ch_lib_from_txt=pd.concat([gen_ch_lib_desc[['label','category','td001_dpe_id']],
                               gen_ch_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ch_lib_from_txt =gen_ch_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ch_lib_from_txt = gen_ch_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')
# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
gen_ch_lib_from_txt.label = gen_ch_lib_from_txt.label.str.replace(' exact','')

gen_ch_lib_from_txt['source']='txt'

# libéllés des générateurs chauffage issus de la table td012
gen_ch_lib_from_data = td012_p[['td001_dpe_id','gen_ch_lib_infer']].copy()

gen_ch_lib_from_data.columns=['td001_dpe_id','label']

gen_ch_lib_from_data['category']=gen_ch_lib_from_data.label.replace(reverse_cat_gen_ch)

gen_ch_lib_from_data['label']=pd.Categorical(gen_ch_lib_from_data.label,categories=list(gen_ch_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ch_lib_from_data['source']='data'

# concat. 
gen_ch_lib = pd.concat([gen_ch_lib_from_data,gen_ch_lib_from_txt],axis=0)

gen_ch_lib['source']=pd.Categorical(gen_ch_lib.source,categories=['data','txt'],ordered=True)
gen_ch_lib['category']=pd.Categorical(gen_ch_lib.category,categories=list(gen_ch_search_dict.keys()),ordered=True)



#### type installation

In [398]:


# libéllés des generateurs chauffage issues des données textes
type_installation_ch_from_txt = pd.concat([type_installation_ch_desc[['label','td001_dpe_id']],
                               type_installation_ch_ft[['label','td001_dpe_id']]],axis=0)

type_installation_ch_from_txt['label'] = pd.Categorical(type_installation_ch_from_txt['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)



type_installation_ch_from_txt = type_installation_ch_from_txt.sort_values(by=['td001_dpe_id','label'])

type_installation_ch_from_txt = type_installation_ch_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

type_installation_ch_from_txt['source'] = 'txt'

type_installation_ch_from_data = td011_p[['td001_dpe_id','type_installation_ch']].copy()
type_installation_ch_from_data.columns = ['td001_dpe_id','label']
type_installation_ch_from_data['label'] = type_installation_ch_from_data['label'].fillna('indetermine')
type_installation_ch_from_data['label'] = pd.Categorical(type_installation_ch_from_data['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)
type_installation_ch_from_data['source'] = 'data'
type_installation_ch = pd.concat([type_installation_ch_from_data,type_installation_ch_from_txt],axis=0)


type_installation_ch['is_indetermine']=type_installation_ch.label=='indetermine'

#### energie

In [399]:


# libéllés des generateurs chauffage issues des données textes
energie_ch_from_txt = pd.concat([energie_ch_desc[['label','td001_dpe_id']],
                               energie_ch_ft[['label','td001_dpe_id']]],axis=0)

energie_ch_from_txt['label'] = pd.Categorical(energie_ch_from_txt['label'],
                                                    categories=energie_normalise_ordered,ordered=True)



energie_ch_from_txt = energie_ch_from_txt.sort_values(by=['td001_dpe_id','label'])

energie_ch_from_txt = energie_ch_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

energie_ch_from_txt['source'] = 'txt'

energie_ch_from_data = td012_p[['td001_dpe_id','type_energie_ch']].copy()
energie_ch_from_data.columns = ['td001_dpe_id','label']
energie_ch_from_data['label'] = energie_ch_from_data['label'].fillna('indetermine')
energie_ch_from_data['label'] = pd.Categorical(energie_ch_from_data['label'],
                                                    categories=energie_normalise_ordered,ordered=True)
energie_ch_from_data['source'] = 'data'
energie_ch = pd.concat([energie_ch_from_data,energie_ch_from_txt],axis=0)


energie_ch['is_indetermine']=energie_ch.label=='indetermine'

### amélioration des labels chaudières

Problème : dans les données ou le texte on manque parfois de l'information type energie et/ou type de chaudiere (condensation/BT etc..) on cherche le maximum d'information des diverses sources pour recréer le bon libéllé

In [400]:
is_chaudiere_ener_ind = gen_ch_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ch_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ch_lib.label.str.contains('chaudiere')&gen_ch_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


gen_ch_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ch_lib['chaudiere_ind']=is_chaudiere_ind

sum_inds=gen_ch_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

ind_sel = gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ind
chaudiere_ind=gen_ch_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ch_lib.loc[gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ener_ind]

suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

gen_ch_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

### fusion

#### generateur chauffage

In [401]:
gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])


gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ch_lib = gen_ch_lib.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ch_lib=gen_ch_lib.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ch_lib = gen_ch_lib.to_frame('gen_ch_lib_final')

td001_sys = td001_sys.merge(gen_ch_lib.reset_index(),on='td001_dpe_id',how='left')

diff=td001_sys.gen_ch_lib_infer_concat!=td001_sys.gen_ch_lib_final

ind=td001_sys.loc[td001_sys.gen_ch_lib_final=='chaudiere energie indetermine condensation']

#### chauffage solaire

In [402]:
solaire_ch_txt=pd.concat([solaire_ch_ft[['label','td001_dpe_id']],solaire_ch_desc[['label','td001_dpe_id']]],axis=0)

solaire_ch_txt = solaire_ch_txt.sort_values('label').drop_duplicates(subset=['td001_dpe_id'])

solaire_ch_txt= solaire_ch_txt.loc[solaire_ch_txt.label=='solaire']

is_ch_solaire_txt=solaire_ch_txt.groupby('td001_dpe_id').label.count().astype(float)>0

td011_p['tr003_solaire'] = td011_p['tr003_solaire'].astype(float)

is_ch_solaire_data = td011_p.groupby('td001_dpe_id').tr003_solaire.max()>0

td001_sys = td001_sys.merge(is_ch_solaire_data.to_frame('is_ch_solaire_data'),on='td001_dpe_id',how='left')

td001_sys = td001_sys.merge(is_ch_solaire_txt.to_frame('is_ch_solaire_txt'),on='td001_dpe_id',how='left')

td001_sys['is_ch_solaire'] = td001_sys.is_ch_solaire_txt.fillna(0).astype(float)+td001_sys.is_ch_solaire_data.fillna(0).astype(float)
td001_sys['is_ch_solaire'] = td001_sys['is_ch_solaire'] > 0

is_ch_solaire = td001_sys.is_ch_solaire==True

td001_sys.loc[is_ch_solaire,'gen_ch_lib_final']='chauffage solaire + '+td001_sys.loc[is_ch_solaire,'gen_ch_lib_final']

#### type installation

In [403]:
type_installation_ch = type_installation_ch.sort_values(by=['td001_dpe_id','is_indetermine',
                                     'source','label'])
type_installation_ch = type_installation_ch.drop_duplicates(subset=['td001_dpe_id'])

type_installation_ch = type_installation_ch.set_index('td001_dpe_id').label.to_frame('type_installation_ch')

td001_sys = td001_sys.merge(type_installation_ch.reset_index(),on='td001_dpe_id',how='left')


#### energie

In [404]:

energie_ch = energie_ch.sort_values(by=['td001_dpe_id','label','source'])

energie_ch = energie_ch.drop_duplicates(subset=['td001_dpe_id','label'])

energie_ch=energie_ch.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x))).to_frame('type_energie_ch')




td001_sys = td001_sys.merge(energie_ch.reset_index(),on='td001_dpe_id',how='left')


#  ecs

### preparation de la donnée


#### generateur ecs

In [405]:


# libéllés des generateurs chauffage issues des données textes
gen_ecs_lib_from_txt=pd.concat([gen_ecs_lib_desc[['label','category','td001_dpe_id']],
                               gen_ecs_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ecs_lib_from_txt =gen_ecs_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ecs_lib_from_txt = gen_ecs_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')

# suppression de la dénomination exact qui ne correpond qu'a une méthode de recherche plus rigide
gen_ecs_lib_from_txt.label = gen_ecs_lib_from_txt.label.str.replace(' exact','')

gen_ecs_lib_from_txt['source']='txt'

# libéllés des générateurs chauffage issus de la table td012
gen_ecs_lib_from_data = td014_p[['td001_dpe_id','gen_ecs_lib_infer']].copy()

gen_ecs_lib_from_data.columns=['td001_dpe_id','label']
gen_ecs_lib_from_data['label']=gen_ecs_lib_from_data['label'].fillna('indetermine')
gen_ecs_lib_from_data['category']=gen_ecs_lib_from_data.label.replace(reverse_cat_gen_ecs)

gen_ecs_lib_from_data['label']=pd.Categorical(gen_ecs_lib_from_data.label,categories=list(gen_ecs_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ecs_lib_from_data['label']=gen_ecs_lib_from_data['label'].fillna('indetermine')

gen_ecs_lib_from_data['source']='data'

# concat. 
gen_ecs_lib = pd.concat([gen_ecs_lib_from_data,gen_ecs_lib_from_txt],axis=0)

gen_ecs_lib['source']=pd.Categorical(gen_ecs_lib.source,categories=['data','txt'],ordered=True)
gen_ecs_lib['category']=pd.Categorical(gen_ecs_lib.category,categories=list(gen_ecs_search_dict.keys()),ordered=True)



#### type installation

In [406]:


# libéllés des generateurs chauffage issues des données textes
type_installation_ecs_from_txt = pd.concat([type_installation_ecs_desc[['label','td001_dpe_id']],
                               type_installation_ecs_ft[['label','td001_dpe_id']]],axis=0)

type_installation_ecs_from_txt['label'] = pd.Categorical(type_installation_ecs_from_txt['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)



type_installation_ecs_from_txt = type_installation_ecs_from_txt.sort_values(by=['td001_dpe_id','label'])

type_installation_ecs_from_txt = type_installation_ecs_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

type_installation_ecs_from_txt['source'] = 'txt'

type_installation_ecs_from_data = td014_p[['td001_dpe_id','type_installation_ecs']].copy()
type_installation_ecs_from_data.columns = ['td001_dpe_id','label']
type_installation_ecs_from_data['label'] = type_installation_ecs_from_data['label'].fillna('indetermine')
type_installation_ecs_from_data['label'] = pd.Categorical(type_installation_ecs_from_data['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)
type_installation_ecs_from_data['source'] = 'data'
type_installation_ecs = pd.concat([type_installation_ecs_from_data,type_installation_ecs_from_txt],axis=0)


type_installation_ecs['is_indetermine']=type_installation_ecs.label=='indetermine'

#### energie 

In [407]:


# libéllés des generateurs chauffage issues des données textes
energie_ecs_from_txt = pd.concat([energie_ecs_desc[['label','td001_dpe_id']],
                               energie_ecs_ft[['label','td001_dpe_id']]],axis=0)

energie_ecs_from_txt['label'] = pd.Categorical(energie_ecs_from_txt['label'],
                                                    categories=energie_normalise_ordered,ordered=True)



energie_ecs_from_txt = energie_ecs_from_txt.sort_values(by=['td001_dpe_id','label'])

energie_ecs_from_txt = energie_ecs_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

energie_ecs_from_txt['source'] = 'txt'

energie_ecs_from_data = td014_p[['td001_dpe_id','type_energie_ecs']].copy()
energie_ecs_from_data.columns = ['td001_dpe_id','label']
energie_ecs_from_data['label'] = energie_ecs_from_data['label'].fillna('indetermine')
energie_ecs_from_data['label'] = pd.Categorical(energie_ecs_from_data['label'],
                                                    categories=energie_normalise_ordered,ordered=True)
energie_ecs_from_data['source'] = 'data'
energie_ecs = pd.concat([energie_ecs_from_data,energie_ecs_from_txt],axis=0)


energie_ecs['is_indetermine']=energie_ecs.label=='indetermine'

### amélioration des labels chaudières

Problème : dans les données ou le texte on manque parfois de l'information type energie et/ou type de chaudiere (condensation/BT etc..) on cherche le maximum d'information des diverses sources pour recréer le bon libéllé

In [408]:
is_chaudiere_ener_ind = gen_ecs_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ecs_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ecs_lib.label.str.contains('chaudiere')&gen_ecs_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


gen_ecs_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ecs_lib['chaudiere_ind']=is_chaudiere_ind

sum_inds=gen_ecs_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

ind_sel = gen_ecs_lib.td001_dpe_id.isin(id_sel)&gen_ecs_lib.chaudiere_ind
chaudiere_ind=gen_ecs_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id.isin(id_sel)&gen_ecs_lib.chaudiere_ener_ind]

suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

gen_ecs_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

### fusion

In [409]:
priorisation_ecs={'chaudiere':'principal',
 'effet_joule':'secondaire',
"production_mixte_indetermine":"secondaire",
 'chauffe-eau_independant':'defaut',
 'reseau de chaleur':'principal',
 'solaire':'solaire',
 'abscence_solaire':'solaire',
 'ecs_thermodynamique':'principal',
'indetermine':'defaut'}

In [410]:
gen_ecs_lib['priorite']=gen_ecs_lib.category.replace(priorisation_ecs)

#### traitement des generateurs d'ECS prioritaires

In [411]:
gen_ecs_lib_princ = gen_ecs_lib.query('priorite=="principal"')

In [412]:


gen_ecs_lib_princ = gen_ecs_lib_princ.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_princ = gen_ecs_lib_princ.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_princ=gen_ecs_lib_princ.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_princ = gen_ecs_lib_princ.to_frame('gen_ecs_lib_final')



td001_sys = td001_sys.merge(gen_ecs_lib_princ.reset_index(),on='td001_dpe_id',how='left')


#### traitement des generateurs d'ecs secondaires

In [413]:
gen_ecs_lib_sec = gen_ecs_lib.query('priorite=="secondaire"')

gen_ecs_lib_sec = gen_ecs_lib_sec.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_sec = gen_ecs_lib_sec.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_sec=gen_ecs_lib_sec.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_sec = gen_ecs_lib_sec.to_frame('gen_ecs_lib_final_sec')

td001_sys = td001_sys.merge(gen_ecs_lib_sec.reset_index(),on='td001_dpe_id',how='left')

null=td001_sys.gen_ecs_lib_final.isnull()

td001_sys.loc[null,'gen_ecs_lib_final']=td001_sys.loc[null,'gen_ecs_lib_final_sec']

#### traitement des generateurs d'ecs par défaut

In [414]:
gen_ecs_lib_defaut = gen_ecs_lib.query('priorite=="defaut"')

gen_ecs_lib_defaut = gen_ecs_lib_defaut.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_defaut = gen_ecs_lib_defaut.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_defaut=gen_ecs_lib_defaut.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_defaut = gen_ecs_lib_defaut.to_frame('gen_ecs_lib_final_defaut')

td001_sys = td001_sys.merge(gen_ecs_lib_defaut.reset_index(),on='td001_dpe_id',how='left')

null=td001_sys.gen_ecs_lib_final.isnull()

td001_sys.loc[null,'gen_ecs_lib_final']=td001_sys.loc[null,'gen_ecs_lib_final_defaut']

####  ecs solaire

In [415]:
gen_ecs_lib_solaire = gen_ecs_lib.query('priorite=="solaire"')


gen_ecs_lib_solaire =gen_ecs_lib_solaire.sort_values('label',ascending=False).drop_duplicates(subset='td001_dpe_id')

gen_ecs_lib_solaire=gen_ecs_lib_solaire.loc[gen_ecs_lib_solaire.label=='ecs solaire']

is_ecs_solaire_txt=(gen_ecs_lib_solaire.groupby('td001_dpe_id').label.count().astype(float)>0)

is_ecs_solaire_data=td014_p.groupby('td001_dpe_id').is_ecs_solaire.max().astype(float)>0

td001_sys = td001_sys.merge(is_ecs_solaire_txt.to_frame('is_ecs_solaire_txt'),how='left',on='td001_dpe_id')
td001_sys = td001_sys.merge(is_ecs_solaire_data.to_frame('is_ecs_solaire_data'),how='left',on='td001_dpe_id')


td001_sys['is_ecs_solaire'] = td001_sys.is_ecs_solaire_txt.fillna(0).astype(float)+td001_sys.is_ecs_solaire_data.fillna(0).astype(float)
td001_sys['is_ecs_solaire'] = td001_sys['is_ecs_solaire'] > 0




is_ecs_solaire = td001_sys.is_ecs_solaire==True

td001_sys.loc[is_ecs_solaire,'gen_ecs_lib_final']='ecs solaire + '+td001_sys.loc[is_ecs_solaire,'gen_ecs_lib_final']

####  installation ecs

In [416]:
type_installation_ecs = type_installation_ecs.sort_values(by=['td001_dpe_id','is_indetermine',
                                     'source','label'])
type_installation_ecs = type_installation_ecs.drop_duplicates(subset=['td001_dpe_id'])

type_installation_ecs = type_installation_ecs.set_index('td001_dpe_id').label.to_frame('type_installation_ecs')

td001_sys = td001_sys.merge(type_installation_ecs.reset_index(),on='td001_dpe_id',how='left')


####  energie ecs

In [417]:

energie_ecs = energie_ecs.sort_values(by=['td001_dpe_id','label','source'])

energie_ecs = energie_ecs.drop_duplicates(subset=['td001_dpe_id','label'])

energie_ecs=energie_ecs.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x))).to_frame('type_energie_ecs')




td001_sys = td001_sys.merge(energie_ecs.reset_index(),on='td001_dpe_id',how='left')


# Redressement des données 

In [418]:
td001_sys['gen_ch_lib_final']=td001_sys.gen_ch_lib_final.str.replace(' \+ indetermine','')
td001_sys['gen_ecs_lib_final']=td001_sys.gen_ecs_lib_final.str.replace(' \+ indetermine','')

In [419]:
td001_sys['gen_ch_lib_non_retraite']=td001_sys.gen_ch_lib_final.copy()
td001_sys['gen_ecs_lib_non_retraite']=td001_sys.gen_ecs_lib_final.copy()

## retraitement des energies non affectées pour les chaudières avec les energies factures

In [420]:
td001_sys =td001_sys.merge(td001_ener_from_facture,on='td001_dpe_id',how='left')


### chauffage

In [421]:


ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')

for energie in energie_chaudiere_mods:
    
    # ON AFFECTE LES ENERGIES MANQUANTES AVEC LES ENERGIES DE CHAUFFAGE ISSUES DIFFERENTES SOURCES
    
    ener = td001_sys.td002_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td016_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    # SINON ON PREND LES ENERGIES D'ECS ISSUES DES DIFFERENTES SOURCES

    ener = td001_sys.type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td002_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td016_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')


### ECS

In [422]:


ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')

for energie in energie_chaudiere_mods:
    
    # ON AFFECTE LES ENERGIES MANQUANTES AVEC LES ENERGIES D'ECS ISSUES DIFFERENTES SOURCES

    ener = td001_sys.td002_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    ener = td001_sys.td016_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    # SINON ON PREND LES ENERGIES DE CHAUFFAGE ISSUES DES DIFFERENTES SOURCES

    ener = td001_sys.type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td002_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    
    ener = td001_sys.td016_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')


## cohérence sys ecs chauffage

In [423]:
td001_sys[['gen_ch_lib_final','gen_ecs_lib_final']]=td001_sys[['gen_ch_lib_final','gen_ecs_lib_final']].fillna('indetermine')

In [424]:
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

### chaudieres

In [425]:
is_chaudiere_ener_ind_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('chaudiere energie indetermine')
is_chaudiere_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere')
is_chaudiere_ecs = td001_sys.gen_ecs_lib_final.str.contains('chaudiere')



#### affectation du système mixte à l'ECS lorsque celle ci est indetermine

In [426]:
sub_by_ch=is_ind_ecs&is_chaudiere_ch

In [427]:
td001_sys.loc[sub_by_ch,'gen_ecs_lib_final']=td001_sys.loc[sub_by_ch,'gen_ch_lib_final']

#### correction et convergence des energies de chaudieres entre ECS et chauffage

In [428]:
for energie in list(reversed(energie_chaudiere_mods)):
    is_ener_ch = (td001_sys.gen_ch_lib_final.str.contains(energie))
    is_ener_ecs = (td001_sys.gen_ecs_lib_final.str.contains(energie))
    sub_ch_ecs = is_ener_ch&(~is_ener_ecs)&is_chaudiere_ch&is_chaudiere_ecs
    td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final']=td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    sub_ecs_ch = is_ener_ecs&(~is_ener_ch)&is_chaudiere_ch&is_chaudiere_ecs
    td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final']=td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final'].str.replace('energie indetermine',energie)


#### correction et convergence des types de chaudieres entre ECS et chauffage

In [429]:
for energie in reversed(energie_chaudiere_mods):
    is_ener_ch = (td001_sys.gen_ch_lib_final.str.contains(energie))
    is_ener_ecs = (td001_sys.gen_ecs_lib_final.str.contains(energie))
    both_ener = is_ener_ch&is_ener_ecs&is_chaudiere_ch&is_chaudiere_ecs
    for type_chaudiere in reversed(type_chaudiere_mods):
        is_type_ch = (td001_sys.gen_ch_lib_final.str.contains(type_chaudiere))
        is_type_ecs = (td001_sys.gen_ecs_lib_final.str.contains(type_chaudiere))
        chaudiere_ind = f'chaudiere {energie} indetermine'
        chaudiere_type=f'chaudiere {energie} {type_chaudiere}'
        is_ind_ch = (td001_sys.gen_ch_lib_final.str.contains(chaudiere_ind))
        is_ind_ecs = (td001_sys.gen_ecs_lib_final.str.contains(chaudiere_ind))
        sub_ch_ecs = is_type_ch&both_ener&is_ind_ecs
        td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final']=td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final'].str.replace(chaudiere_ind,chaudiere_type)
        sub_ecs_ch = is_type_ecs&both_ener&is_ind_ch
        td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final']=td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final'].str.replace(chaudiere_ind,chaudiere_type)

        

#### correction cas ECS affecté à un chauffe eau indépendant lorsque chaudiere chauffage

In [430]:
# refresh
is_chaudiere_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere')
is_ind = td001_sys.gen_ecs_lib_final.str.contains('chauffe-eau independant indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final=='indetermine')

In [431]:
td001_sys.loc[is_chaudiere_ch&is_ind,'gen_ecs_lib_final']=td001_sys.loc[is_chaudiere_ch&is_ind,'gen_ch_lib_final']

#### correction cas ecs si gen_ch_lib elec ou poele

In [432]:
# si le generateur de chauffage est décentralisé et avec une energie non combustible. 
is_ind = td001_sys.gen_ecs_lib_final.str.contains('chauffe-eau independant indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final=='indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine'))

In [433]:
is_ch_decentr = ~td001_sys.gen_ch_lib_final.str.contains('|'.join(energie_chaudiere_mods))
is_ch_decentr = is_ch_decentr& td001_sys.gen_ch_lib_final.str.contains('elec|pac|joule|jonction|poele')

In [434]:
td001_sys.loc[is_ind&is_ch_decentr,'gen_ecs_lib_final']='ecs electrique indetermine'

In [435]:
is_ind =(td001_sys.gen_ecs_lib_final=='indetermine')

### reseaux de chaleur

In [436]:
# refresh
# si le chauffage ou l'ECS sont indéterminés et que l'autre est un réseau de chaleur alors on substitue par réseau de chaleur
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

In [437]:
is_reseau_ch = td001_sys.gen_ch_lib_final.str.contains('reseau de chaleur')
is_reseau_ecs = td001_sys.gen_ecs_lib_final.str.contains('reseau de chaleur')

td001_sys.loc[is_ind_ch&is_reseau_ecs,'gen_ch_lib_final']='reseau de chaleur'
td001_sys.loc[is_ind_ecs&is_reseau_ch,'gen_ecs_lib_final']='reseau de chaleur'

### thermodynamique

In [438]:
# refresh

# si l'un est thermodynamique et l'autre indeterminé on considère l'autre comme thermodynamique. 
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

In [439]:
is_pac_ch = td001_sys.gen_ch_lib_final.str.contains('pac')&(~td001_sys.gen_ch_lib_final.str.contains('pac air/air'))

td001_sys.loc[is_ind_ecs&is_pac_ch,'gen_ecs_lib_final']='ecs thermodynamique electrique(pac ou ballon)'

### pac en releve de chaudiere

In [440]:
# si on a une chaudiere et une pac alors on considère pac en relève de chaudière
is_pac_ch = td001_sys.gen_ch_lib_final.str.contains('pac')&(~td001_sys.gen_ch_lib_final.str.contains('pac air/air'))
is_pac_ch = is_pac_ch& (td001_sys.gen_ch_lib_final !='pac indetermine')
is_chaudiere_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere')

for pac_type in ['pac air/eau','pac eau/eau','pac geothermique']:
    
    is_pac_ch = td001_sys.gen_ch_lib_final.str.contains(pac_type)
    
    td001_sys.loc[is_pac_ch&is_chaudiere_ch,'gen_ch_lib_final']=f'{pac_type} en releve de chaudiere'
    

## affectation de libéllés par défaut en fonction des energies de chauffage et ecs pour les générateurs indéterminés

In [441]:
ind = td001_sys.gen_ch_lib_final=="indetermine"

In [442]:
for type_ener in td001_sys.td002_type_energie_ch.dropna().unique():
    
    ener = td001_sys.td002_type_energie_ch==type_ener
    
    if type_ener in energie_chaudiere_mods:
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'chaudiere {type_ener} indetermine'
        
    elif 'reseau de chaleur' in type_ener : 
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'reseau de chaleur'
        
    else:
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'chauffage indetermine {type_ener}'

for type_ener in td001_sys.td016_type_energie_ch.dropna().unique():

    ener = td001_sys.td016_type_energie_ch==type_ener
    
    if type_ener in energie_chaudiere_mods:
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'chaudiere {type_ener} indetermine'
        
    elif 'reseau de chaleur' in type_ener : 
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'reseau de chaleur'
        
    else:
        
        td001_sys.loc[ind&ener,'gen_ch_lib_final']=f'chauffage indetermine {type_ener}'

    

# indicateurs de cohérences

In [462]:
lib_final_eq_lib_data = (td001_sys.gen_ch_lib_infer_concat==td001_sys.gen_ch_lib_final).fillna(False)

td001_sys['fiab_gen_ch_lib'] = 'libelle non redresse determine'

ind_data=td001_sys.gen_ch_lib_infer_concat.str.contains('indetermine').fillna(False)
not_data = td001_sys.gen_ch_lib_infer_concat.isnull()
td001_sys.loc[lib_final_eq_lib_data&(~ind_data),'fiab_gen_ch_lib']='libelle fiable et coherent ft desc et td012'

is_redresse = td001_sys.gen_ch_lib_final!=td001_sys.gen_ch_lib_non_retraite
td001_sys.loc[is_redresse,'fiab_gen_ch_lib'] = 'libelle redresse determine'

is_ind = td001_sys.gen_ch_lib_final=='indetermine'

td001_sys.loc[is_ind,'fiab_gen_ch_lib'] = 'libelle indetermine'

is_ind = td001_sys.gen_ch_lib_final.str.contains('indetermine')

td001_sys.loc[is_ind,'fiab_gen_ch_lib'] = td001_sys.loc[is_ind,'fiab_gen_ch_lib'].str.replace(' determine',' partiellement indetermine')



In [466]:
lib_final_eq_lib_data = (td001_sys.gen_ecs_lib_infer_concat==td001_sys.gen_ecs_lib_final).fillna(False)

td001_sys['fiab_gen_ecs_lib'] = 'libelle non redresse determine'

ind_data=td001_sys.gen_ecs_lib_infer_concat.str.contains('indetermine').fillna(False)
not_data = td001_sys.gen_ecs_lib_infer_concat.isnull()
td001_sys.loc[lib_final_eq_lib_data&(~ind_data),'fiab_gen_ecs_lib']='libelle fiable et coherent ft desc et td012'

is_redresse = td001_sys.gen_ecs_lib_final!=td001_sys.gen_ecs_lib_non_retraite
td001_sys.loc[is_redresse,'fiab_gen_ecs_lib'] = 'libelle redresse determine'

is_ind = td001_sys.gen_ecs_lib_final=='indetermine'

td001_sys.loc[is_ind,'fiab_gen_ecs_lib'] = 'libelle indetermine'

is_ind = td001_sys.gen_ecs_lib_final.str.contains('indetermine')

td001_sys.loc[is_ind,'fiab_gen_ecs_lib'] = td001_sys.loc[is_ind,'fiab_gen_ecs_lib'].str.replace(' determine',' partiellement indetermine')



# nettoyage final

## suppression des additions d'indeterminé

# simplification 

## sys ch

In [468]:
td001_sys['gen_ch_lib_final_simp']=td001_sys['gen_ch_lib_final']

### simplification des dénominations generateurs

In [474]:
gen_ch_simp_dict = {
                        'convecteurs electriques nfc': 'generateurs a effet joule',
                        'panneaux rayonnants electriques nfc': 'generateurs a effet joule',
                        'radiateurs electriques': 'generateurs a effet joule',
                        'autres emetteurs a effet joule': 'generateurs a effet joule',
                        'plafonds/planchers rayonnants electriques nfc': 'generateurs a effet joule',
                        'convecteurs bi-jonction': 'generateurs a effet joule',}

for ener in energie_chaudiere_mods:
    
    gen_ch_simp_dict[f'chaudiere {ener} indetermine']=f'chaudiere {ener} standard'

In [475]:
for k,v in gen_ch_simp_dict.items():
    td001_sys.gen_ch_lib_final_simp=td001_sys.gen_ch_lib_final_simp.str.replace(k,v)
    

### simplification réseau de chaleur
si réseau de chaleur alors on considère que c'est le système de chauffage du bâtiment

In [477]:
is_reseau_ch = td001_sys.gen_ch_lib_final.str.contains('reseau de chaleur')

td001_sys.loc[is_reseau_ch,'gen_ch_lib_final_simp']='reseau de chaleur'

### simplifications appoints divers

In [478]:
ind = td001_sys.loc[td001_sys.gen_ch_lib_final_simp=='indetermine']

In [710]:
ind[['td002_type_energie_ch','td002_type_energie_ecs','td016_type_energie_ch','td016_type_energie_ecs']].count()

td002_type_energie_ch     9174
td002_type_energie_ecs    7797
td016_type_energie_ch      844
td016_type_energie_ecs     593
dtype: int64

In [702]:
td001_sys.gen_ch_lib_final_simp.value_counts().to_dict()

{'chaudiere gaz standard': 35219,
 'generateurs a effet joule': 34533,
 'reseau de chaleur': 25977,
 'chaudiere gaz condensation': 18016,
 'indetermine': 11609,
 'chaudiere gaz basse temperature': 6332,
 'chaudiere fioul standard': 5544,
 'chaudiere energie indetermine standard': 1172,
 'pac air/air': 736,
 'chaudiere energie indetermine indetermine': 475,
 'pac air/eau': 462,
 'chaudiere fioul basse temperature': 451,
 'chaudiere gaz condensation + generateurs a effet joule': 442,
 'chaudiere gaz standard + generateurs a effet joule': 383,
 'generateurs a effet joule + chaudiere electrique': 371,
 'generateurs a effet joule + pac air/air': 349,
 'chaudiere energie indetermine condensation': 311,
 'pac eau/eau en releve de chaudiere': 231,
 'generateurs a effet joule + poele ou insert bois': 222,
 'chaudiere gaz condensation + poele ou insert bois': 213,
 'chaudiere electrique + generateurs a effet joule': 200,
 'chaudiere gaz standard + poele ou insert bois': 178,
 'poele ou insert bo

In [684]:
td001_sys.loc[td001_sys.gen_ch_lib_final_simp=="radiateurs gaz"]

,td001_dpe_id,tr002_type_batiment_id,gen_ch_lib_infer_concat,gen_ecs_lib_infer_concat,gen_ch_lib_final,is_ch_solaire_data,is_ch_solaire_txt,is_ch_solaire,type_installation_ch,type_energie_ch,...,is_ecs_solaire_data,is_ecs_solaire,type_installation_ecs,type_energie_ecs,td002_type_energie_ch,td002_type_energie_ecs,td016_type_energie_ch,td016_type_energie_ecs,gen_ch_lib,gen_ch_lib_final_simp
2021,122603,4,NaN,NaN,radiateurs gaz,NaN,NaN,False,NaN,gaz,...,NaN,False,NaN,gaz,gaz,NaN,NaN,NaN,NaN,radiateurs gaz
2760,164455,2,NaN,NaN,radiateurs gaz,NaN,NaN,False,NaN,gaz,...,NaN,False,NaN,electricite,NaN,NaN,NaN,NaN,NaN,radiateurs gaz
5088,337006,2,NaN,NaN,radiateurs gaz,NaN,NaN,False,NaN,gaz,...,NaN,False,NaN,gaz,NaN,NaN,gaz,gaz,NaN,radiateurs gaz
7223,457859,1,NaN,NaN,radiateurs gaz,NaN,NaN,False,NaN,gaz,...,NaN,False,individuel,electricite,gaz,electricite,gaz,electricite,NaN,radiateurs gaz
7898,501911,1,NaN,NaN,radiateurs gaz,NaN,NaN,False,NaN,gaz,...,NaN,False,individuel,electricite,gaz,NaN,gaz,NaN,NaN,radiateurs gaz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137527,8813157,2,NaN,NaN,radiateurs gaz,NaN,NaN,False,individuel,gaz,...,NaN,False,individuel,electricite,NaN,NaN,NaN,NaN,NaN,radiateurs gaz
142375,9164001,1,NaN,NaN,radiateurs gaz,NaN,NaN,False,individuel,gaz,...,NaN,False,individuel,electricite,NaN,NaN,NaN,NaN,NaN,radiateurs gaz
143887,9264048,2,NaN,NaN,radiateurs gaz,NaN,NaN,False,individuel,gaz,...,NaN,False,individuel,NaN,gaz,NaN,NaN,NaN,NaN,radiateurs gaz
145063,9343853,2,NaN,NaN,radiateurs gaz,NaN,NaN,False,individuel,gaz,...,NaN,False,individuel,NaN,NaN,NaN,NaN,NaN,NaN,radiateurs gaz


In [692]:
td001_sys.loc[td001_sys.gen_ch_lib_final_simp=='generateurs a effet joule + reseau de chaleur']

,td001_dpe_id,tr002_type_batiment_id,gen_ch_lib_infer_concat,gen_ecs_lib_infer_concat,gen_ch_lib_final,is_ch_solaire_data,is_ch_solaire_txt,is_ch_solaire,type_installation_ch,type_energie_ch,...,is_ecs_solaire_data,is_ecs_solaire,type_installation_ecs,type_energie_ecs,td002_type_energie_ch,td002_type_energie_ecs,td016_type_energie_ch,td016_type_energie_ecs,gen_ch_lib,gen_ch_lib_final_simp
5603,375290,4,NaN,NaN,convecteurs electriques nfc + reseau de chaleur,NaN,NaN,False,collectif,electricite,...,NaN,False,collectif,electricite,autre + electricite,autre + electricite,electricite + reseau de chaleur,electricite + reseau de chaleur,NaN,generateurs a effet joule + reseau de chaleur
10391,666458,4,NaN,NaN,autres emetteurs a effet joule + reseau de cha...,NaN,NaN,False,individuel,electricite,...,NaN,False,NaN,electricite,NaN,NaN,NaN,NaN,NaN,generateurs a effet joule + reseau de chaleur
16603,1118244,2,NaN,NaN,convecteurs electriques nfc + reseau de chaleur,NaN,NaN,False,collectif,electricite,...,NaN,False,collectif,NaN,autre + electricite,autre,electricite + reseau de chaleur,reseau de chaleur,NaN,generateurs a effet joule + reseau de chaleur
19199,1297778,4,NaN,NaN,plafonds/planchers rayonnants electriques nfc ...,NaN,NaN,False,NaN,NaN,...,NaN,False,NaN,electricite,autre,autre + electricite,NaN,NaN,NaN,generateurs a effet joule + reseau de chaleur
20672,1378897,2,NaN,NaN,radiateurs electriques + reseau de chaleur,NaN,NaN,False,NaN,electricite,...,NaN,False,individuel,bois,autre + electricite,NaN,electricite,NaN,NaN,generateurs a effet joule + reseau de chaleur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141946,9127953,4,NaN,NaN,radiateurs electriques + reseau de chaleur,NaN,NaN,False,NaN,electricite,...,NaN,False,NaN,electricite,autre + electricite,NaN,NaN,NaN,NaN,generateurs a effet joule + reseau de chaleur
141947,9127964,4,NaN,NaN,radiateurs electriques + reseau de chaleur,NaN,NaN,False,NaN,electricite,...,NaN,False,NaN,electricite,autre + electricite,NaN,NaN,NaN,NaN,generateurs a effet joule + reseau de chaleur
141948,9128006,4,NaN,NaN,radiateurs electriques + reseau de chaleur,NaN,NaN,False,NaN,electricite,...,NaN,False,NaN,electricite,autre + electricite,NaN,NaN,NaN,NaN,generateurs a effet joule + reseau de chaleur
142050,9136712,4,NaN,NaN,radiateurs electriques + reseau de chaleur,NaN,NaN,False,NaN,electricite,...,NaN,False,NaN,electricite,autre + electricite,NaN,NaN,NaN,NaN,generateurs a effet joule + reseau de chaleur


# debug

In [ ]:
ids=td001_sys.query('gen_ecs_lib_final=="chaudiere energie indetermine standard + reseau de chaleur"').td001_dpe_id.unique()

gen_ecs_lib_from_txt=pd.concat([gen_ecs_lib_desc[['label','category','td001_dpe_id']],
                               gen_ecs_lib_ft[['label','category','td001_dpe_id']]],axis=0)




m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_search_dict_flat)
m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')

gen_ecs_lib_ft

td001_sys.loc[td001_sys.td001_dpe_id=="8085231"]

gen_ecs_lib_ft.loc[gen_ecs_lib_ft.td001_dpe_id=='8085231']

gen_ecs_lib_from_txt.loc[gen_ecs_lib_from_txt.td001_dpe_id=='8085231']

gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id=='8085231']

td005_ch.loc[td005_ch.td001_dpe_id.isin(ids)].to_dict(orient='records')

td003_ch.loc[td003_ch.td001_dpe_id.isin(ids)].to_dict(orient='records')

td003_ecs.loc[td003_ecs.td001_dpe_id.isin(ids)].to_dict(orient='records')

td005_ecs.loc[td005_ecs.td001_dpe_id.isin(ids)].to_dict(orient='records')

td001_sys.gen_ecs_lib_final.fillna('NA').value_counts()

In [87]:
td001_sys.loc[ind,'gen_ch_lib_final'].value_counts()

indetermine                                                2014
chauffage indetermine electricite + gaz                    1604
chauffage indetermine electricite                           896
chaudiere gaz indetermine                                   643
chauffage indetermine autre                                 273
chauffage indetermine bois + electricite                    270
chauffage indetermine bois + electricite + gaz              237
chauffage indetermine bois                                  139
chauffage indetermine autre + electricite                    96
chaudiere fioul indetermine                                  52
reseau de chaleur                                            21
chauffage indetermine autre + bois + electricite             10
chauffage indetermine autre + bois                            9
chaudiere autre : gpl butane propane indetermine              4
chauffage indetermine electricite + fioul                     2
chauffage indetermine bois + fioul      

In [119]:
dpe_list=td001_sys.loc[td001_sys.gen_ch_lib_final=='indetermine'].td001_dpe_id.tolist()

In [291]:
dpe_list = gen_ch_lib_from_txt.loc[gen_ch_lib_from_txt.label=='chaudiere bois'].td001_dpe_id.tolist()

In [482]:
dpe_list = td001_sys.loc[td001_sys.gen_ch_lib_final_simp== 'generateurs a effet joule + chaudiere bois + poele ou insert bois'].td001_dpe_id.tolist()

In [502]:
    td001, td006, td011, td012, td013, td014 = merge_td001_dpe_id_system(td001, td006, td011, td012, td013, td014)

In [491]:
td012_p.loc[td012_p.td001_dpe_id.isin(dpe_list)].columns.tolist()

['td012_generateur_chauffage_id',
 'td001_dpe_id',
 'tr004_code',
 'tr004_description',
 'tv045_code',
 'tv045_energie',
 'tv045_methode',
 'tv045_type_production',
 'tv045_taux_conversion',
 'tv045_conversion_pci_pcs',
 'tv046_code',
 'tv046_nom_reseau',
 'tv046_localisation',
 'tv046_co2',
 'tv046_date_debut',
 'tv046_date_fin',
 'tv016_code',
 'tv016_departement',
 'tv016_altmin',
 'tv016_altmax',
 'tv016_nref',
 'tv016_dhref',
 'tv016_pref',
 'tv016_c2',
 'tv016_c3',
 'tv016_c4',
 'tv016_t_ext_basse',
 'tv016_e',
 'tv016_fch',
 'tv016_fecs_ancienne_m_i',
 'tv016_fecs_recente_m_i',
 'tv016_fecs_solaire_m_i',
 'tv016_fecs_ancienne_i_c',
 'tv016_fecs_recente_i_c',
 'tv017_code',
 'tv017_t_ext_moyen',
 'tv017_peta_cw',
 'tv017_dh14',
 'tv017_prs1',
 'tv018_code',
 'tv018_sclim_inf_150',
 'tv018_sclim_sup_150',
 'tv018_rclim_autres_etages',
 'tv018_rclim_dernier_etage',
 'tv046_type_production',
 'tv028_code',
 'tv028_type_emetteur',
 'tv028_re',
 'tv029_code',
 'tv029_type_distribution

In [495]:
from generate_dpe_annexes_scripts.utils import merge_without_duplicate_columns

In [500]:
td012_p[['td001_dpe_id']]

,td001_dpe_id
0,291
1,493
2,493
3,708
4,1466
...,...
108095,9430979
108096,9431119
108097,9431307
108098,9431405


,id,systeme_chauffage_cogeneration_id,td011_installation_chauffage_id,tr004_type_energie_id,tv045_conversion_kwh_co2_id,tv046_evaluation_contenu_co2_reseaux_id,rendement_emission_systeme_chauffage,tv028_rendement_emission_systeme_chauffage_id,rendement_distribution_systeme_chauffage,tv029_rendement_distribution_systeme_chauffage_id,...,tv034_temperature_fonctionnement_chaudiere_100_id,tv035_temperature_fonctionnement_chaudiere_30_id,rpn,rpint,qp0,puissance_veilleuse,tv036_puissance_veilleuse_id,puissance_nominale,tv038_puissance_nominale_id,consommation_chauffage
0,263,NaN,233,2,8,NaN,0.95,NaN,1.00,NaN,...,NaN,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,7425.01
1,470,NaN,388,2,8,NaN,0.97,NaN,1.00,NaN,...,NaN,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,3844.48
2,471,NaN,389,1,1,NaN,0.95,NaN,1.00,NaN,...,NaN,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,6070.93
3,620,NaN,530,4,3,NaN,0.95,NaN,0.91,NaN,...,NaN,NaN,86.89,84.34,0.28,0.00,NaN,28.00,NaN,37863.52
4,1285,NaN,1119,4,3,NaN,0.95,NaN,0.91,NaN,...,NaN,NaN,82.51,81.77,0.18,0.00,NaN,18.00,NaN,14408.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108095,5364279,NaN,4954352,1,1,NaN,0.95,NaN,1.00,NaN,...,NaN,NaN,0.00,0.00,0.00,0.00,NaN,0.00,NaN,25983.22
108096,5364357,NaN,4954427,3,2,NaN,0.95,NaN,0.91,NaN,...,NaN,NaN,86.76,84.14,0.24,0.00,NaN,24.00,NaN,4350.46
108097,5364464,NaN,4954526,3,2,NaN,0.95,NaN,1.00,NaN,...,NaN,NaN,73.00,73.00,0.00,0.00,NaN,18.00,NaN,4001.35
108098,5364521,NaN,4954579,3,2,NaN,0.95,NaN,0.91,NaN,...,NaN,NaN,86.76,84.14,0.24,0.00,NaN,24.00,NaN,10024.54


In [505]:
from generate_dpe_annexes_scripts.td011_td012_processing import *

In [506]:
table = merge_without_duplicate_columns(td012,td012_p,on='td001_dpe_id')
table = table.astype({k: v for k, v in td012_types.items() if k in table})
table['type_energie_ch'] = table["tv045_energie"].astype('string').fillna('indetermine').replace(
    ener_conv_dict['tv045_energie'])
table['tr004_description'] = table["tr004_description"].astype('string').fillna('indetermine').replace(
    ener_conv_dict['tr004_description'])

is_rpn = table.rpn > 0
is_rpint = table.rpint > 0
is_chaudiere = is_rpint | is_rpn
is_chaudiere = is_chaudiere | ~table.tv038_puissance_nominale_id.isnull()
# all text description raw concat
gen_ch_concat_txt_desc = table['tv031_type_generateur'].astype('string').replace(np.nan, '') + ' '
gen_ch_concat_txt_desc.loc[is_chaudiere] += 'chaudiere '
gen_ch_concat_txt_desc += table['tv036_type_chaudiere'].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["tv030_type_installation"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["tv032_type_generateur"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table['tv035_type_chaudiere'].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table['tv036_type_generation'].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["tv030_type_installation"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["tr004_description"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table["type_energie_ch"].astype('string').replace(np.nan, ' ') + ' '
gen_ch_concat_txt_desc += table['tv046_nom_reseau'].isnull().replace({False: 'réseau de chaleur',
                                                                      True: ""})
gen_ch_concat_txt_desc = gen_ch_concat_txt_desc.str.lower().apply(lambda x: strip_accents(x))

table['gen_ch_concat_txt_desc'] = gen_ch_concat_txt_desc

table['gen_ch_concat_txt_desc'] = table['gen_ch_concat_txt_desc'].apply(lambda x: clean_str(x))

# calcul gen_ch_lib_infer par matching score text.
unique_gen_ch = table.gen_ch_concat_txt_desc.unique()
gen_ch_lib_infer_dict = {k: affect_lib_by_matching_score(k, td012_gen_ch_search_dict) for k in
                         unique_gen_ch}
table['gen_ch_lib_infer'] = table.gen_ch_concat_txt_desc.replace(gen_ch_lib_infer_dict)

In [ ]:
affect_lib_by_matching_score(,td012_gen_ch_search_dict)

In [512]:
txt='bois bois'

In [513]:
lib_dict=td012_gen_ch_search_dict

In [548]:
def compare_(txt, comp):
    if isinstance(comp, tuple):
        count = np.max([txt.count(x) for x in comp])
    else:
        count = txt.count(comp)
    if count > 0:
        return count
    else:
        return -1

comp_score_dict = dict()
for k, v in lib_dict.items():
    comp_score_dict[k] = np.sum([compare_(txt, el) for el in v])

comp = pd.Series(comp_score_dict).to_frame('score').reset_index()
comp['index'] = pd.Categorical(comp['index'],categories=list(lib_dict.keys()),ordered=True)
comp = comp.sort_values(by=['score','index'],ascending=[False,True])
comp = comp[]

,index,score
8,poele ou insert bois,1
10,chaudiere bois,1
11,autres emetteurs a effet joule,-1
5,radiateurs electriques,-2
6,plafonds/planchers rayonnants electriques nfc,-2
9,poele ou insert fioul/gpl/charbon,-2
12,reseau de chaleur,-2
26,chaudiere gaz indetermine,-2
27,chaudiere fioul indetermine,-2
28,chaudiere charbon indetermine,-2


In [531]:
comp_score_dict

{'pac air/air': -3,
 'pac air/eau': -3,
 'pac eau/eau': -3,
 'pac geothermique': -3,
 'panneaux rayonnants electriques nfc': -3,
 'radiateurs electriques': -2,
 'plafonds/planchers rayonnants electriques nfc': -2,
 'convecteurs electriques nfc': -3,
 'poele ou insert bois': 1,
 'poele ou insert fioul/gpl/charbon': -2,
 'chaudiere bois': 1,
 'autres emetteurs a effet joule': -1,
 'reseau de chaleur': -2,
 'convecteurs bi-jonction': -3,
 'chaudiere gaz condensation': -3,
 'chaudiere fioul condensation': -3,
 'chaudiere charbon condensation': -3,
 'chaudiere autre : gpl butane propane condensation': -3,
 'chaudiere gaz basse temperature': -3,
 'chaudiere fioul basse temperature': -3,
 'chaudiere charbon basse temperature': -3,
 'chaudiere autre : gpl butane propane basse temperature': -3,
 'chaudiere gaz standard': -3,
 'chaudiere fioul standard': -3,
 'chaudiere charbon standard': -3,
 'chaudiere autre : gpl butane propane standard': -3,
 'chaudiere gaz indetermine': -2,
 'chaudiere fiou

In [509]:
td012_gen_ch_search_dict

{'pac air/air': ['pac', 'air/air', ('electricite', 'electrique')],
 'pac air/eau': ['pac', 'air/eau', ('electricite', 'electrique')],
 'pac eau/eau': ['pac', 'eau/eau', ('electricite', 'electrique')],
 'pac geothermique': ['pac',
  ('geothermique', 'geothermie'),
  ('electricite', 'electrique')],
 'panneaux rayonnants electriques nfc': ['panneau',
  ('electricite', 'electrique'),
  'nfc'],
 'radiateurs electriques': ['radiateur', ('electricite', 'electrique')],
 'plafonds/planchers rayonnants electriques nfc': [('plancher', 'plafond'),
  ('electricite', 'electrique')],
 'convecteurs electriques nfc': ['convecteur',
  ('electricite', 'electrique'),
  'nfc'],
 'poele ou insert bois': [('poele', 'insert'), ('bois', 'biomasse')],
 'poele ou insert fioul/gpl/charbon': [('poele', 'insert'),
  ('fioul', 'gpl', 'charbon')],
 'chaudiere bois': ['chaudiere', ('bois', 'biomasse')],
 'autres emetteurs a effet joule': [('electricite', 'electrique')],
 'reseau de chaleur': ['reseau', 'chaleur'],
 'c

In [553]:
sel=table.loc[table.td001_dpe_id.isin(dpe_list)&(table.gen_ch_lib_infer=='chaudiere bois')]


In [557]:
sel[['gen_ch_concat_txt_desc']]

,gen_ch_concat_txt_desc
2,bois bois
4,bois bois
38,bois bois
41,bois bois
44,bois bois
...,...
182480,bois bois
182483,bois bois
182489,bois bois
182492,bois bois


In [486]:
td001_sys.loc[td001_sys.td001_dpe_id.isin(dpe_list)]

,td001_dpe_id,tr002_type_batiment_id,gen_ch_lib_infer_concat,gen_ecs_lib_infer_concat,gen_ch_lib_final,is_ch_solaire_data,is_ch_solaire_txt,is_ch_solaire,type_installation_ch,type_energie_ch,...,type_energie_ecs,gen_ch_lib_non_retraite,gen_ecs_lib_non_retraite,td002_type_energie_ch,td002_type_energie_ecs,td016_type_energie_ch,td016_type_energie_ecs,fiab_gen_ch_lib,fiab_gen_ecs_lib,gen_ch_lib_final_simp
3,493,1,autres emetteurs a effet joule + chaudiere bois,ballon a accumulation electrique,autres emetteurs a effet joule + chaudiere boi...,False,NaN,False,individuel,bois + electricite,...,electricite,autres emetteurs a effet joule + chaudiere boi...,ballon a accumulation electrique,bois + electricite,electricite,NaN,NaN,libelle non redresse determine,libelle fiable et coherent ft desc et td012,generateurs a effet joule + chaudiere bois + p...
13,2037,1,autres emetteurs a effet joule + chaudiere bois,ballon a accumulation electrique,autres emetteurs a effet joule + chaudiere boi...,False,NaN,False,individuel,bois + electricite,...,electricite,autres emetteurs a effet joule + chaudiere boi...,ballon a accumulation electrique,bois + electricite,electricite,NaN,NaN,libelle non redresse determine,libelle fiable et coherent ft desc et td012,generateurs a effet joule + chaudiere bois + p...
14,2153,1,autres emetteurs a effet joule + chaudiere bois,ballon a accumulation electrique,autres emetteurs a effet joule + chaudiere boi...,False,NaN,False,individuel,bois + electricite,...,electricite,autres emetteurs a effet joule + chaudiere boi...,ballon a accumulation electrique,bois + electricite,electricite,NaN,NaN,libelle non redresse determine,libelle fiable et coherent ft desc et td012,generateurs a effet joule + chaudiere bois + p...
46,5226,1,autres emetteurs a effet joule + chaudiere bois,ballon a accumulation electrique,autres emetteurs a effet joule + chaudiere boi...,False,NaN,False,individuel,bois + electricite,...,electricite,autres emetteurs a effet joule + chaudiere boi...,ballon a accumulation electrique,bois + electricite,electricite,NaN,NaN,libelle non redresse determine,libelle fiable et coherent ft desc et td012,generateurs a effet joule + chaudiere bois + p...
155,11552,1,autres emetteurs a effet joule + chaudiere bois,ballon a accumulation electrique,autres emetteurs a effet joule + chaudiere boi...,False,NaN,False,individuel,bois + electricite,...,electricite,autres emetteurs a effet joule + chaudiere boi...,ballon a accumulation electrique,bois + electricite,electricite,NaN,NaN,libelle non redresse determine,libelle fiable et coherent ft desc et td012,generateurs a effet joule + chaudiere bois + p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170057,9421941,1,autres emetteurs a effet joule + chaudiere bois,ecs thermodynamique electrique(pompe a chaleur...,autres emetteurs a effet joule + chaudiere boi...,False,NaN,False,individuel,bois + electricite,...,electricite,autres emetteurs a effet joule + chaudiere boi...,ecs solaire + ballon a accumulation electrique,bois + electricite,electricite,NaN,NaN,libelle non redresse determine,libelle non redresse determine,generateurs a effet joule + chaudiere bois + p...
170061,9422087,1,autres emetteurs a effet joule + chaudiere bois,ballon a accumulation electrique,autres emetteurs a effet joule + chaudiere boi...,False,NaN,False,individuel,bois + electricite,...,electricite,autres emetteurs a effet joule + chaudiere boi...,ballon a accumulation electrique,bois + electricite,electricite,NaN,NaN,libelle non redresse determine,libelle fiable et coherent ft desc et td012,generateurs a effet joule + chaudiere bois + p...
170203,9429132,1,autres emetteurs a effet joule + chaudiere bois,ballon a accumulation electrique,autres emetteurs a effet joule + chaudiere boi...,False,NaN,False,individuel,bois + electricite,...,electricite,autres emetteurs a effet joule + chaudiere boi...,ballon a accumulation electrique,bois + electricite,electricite,

In [484]:
a_list =td005_ch.loc[td005_ch.td001_dpe_id.isin(dpe_list)&td005_ch.valeur_renseignee.str.contains('poele')].valeur_renseignee.tolist()


In [485]:
a_list

['poele a granules installe apres 2001<br  re : 1  rr : 0.8  rd : 1  rg : 0.8  pn : 0  fch : 0',
 'poele bois installe apres 2001<br  re : 1  rr : 0.8  rd : 1  rg : 0.8  pn : 0  fch : 0',
 'inst 1 : 3 - installation de chauffage avec insert ou poele bois ou biomasse en appoint',
 'poele installe apres 2001 avec label flamme verte<br  re : 1  rr : 0 8  rd : 1  rg : 0 8  pn : 0  fch : 0',
 'poele installe apres 2001 avec label flamme verte<br  re : 1  rr : 0 8  rd : 1  rg : 0 8  pn : 0  fch : 0',
 'poele installe apres 2001 avec label flamme verte<br  re : 1  rr : 0 8  rd : 1  rg : 0 8  pn : 0  fch : 0',
 'poele installe apres 2001 avec label flamme verte<br  re : 1  rr : 0.8  rd : 1  rg : 0.8  pn : 0  fch : 0',
 'inst 1 : 4 - installation de chauffage par insert  poele bois ou biomasse avec un chauffage electrique dans la sdb',
 'inst 1 : 4 - installation de chauffage par insert  poele bois ou biomasse avec un chauffage electrique dans la sdb',
 'inst 1 : 5 - installation de chauffage a

In [274]:
a_list =td003_ch.loc[td003_ch.td001_dpe_id.isin(dpe_list)].descriptif.tolist()

words = list()
for el in a_list:
    words.extend(el.split(' '))

pd.Series(words).value_counts().to_dict()

sel=td003_ch.loc[td003_ch.td001_dpe_id.isin(dpe_list)]
#sel.descriptif.to_dict()

In [378]:
sel=td005_ch.loc[td005_ch.td001_dpe_id.isin(dpe_list)]

sel.loc[sel.valeur_renseignee.str.contains('collective')].valeur_renseignee.tolist()

[' - chaufferie collective plancher chauffant',
 ' - chaufferie collective gaz ventilo convecteurs 4 tubes  radiateurs eau chaude  aerothermes',
 'generateur pompe a chaleur collective amzair - distribution reparties ventilo-convecteurs dans les parties privatives<br  ',
 'generateur pompe a chaleur collective amzair - distribution reparties ventilo-convecteurs dans les parties privatives<br  ',
 'generateur pompe a chaleur collective amzair - distribution reparties ventilo-convecteurs dans les parties privatives<br  ']

In [388]:
sel.valeur_renseignee.to_dict()

{11228: 'radiateurs nfc avec programmateur<br  ',
 11333: 'installation de chauffage  59 9\xa0m²)\xa0:\r ',
 11668: 'installation de chauffage  57 6\xa0m²)\xa0:\r ',
 15331: 'installation de chauffage  32\xa0m²)\xa0:\r ',
 29848: 'radiateurs nfc<br  ',
 33385: 'radiateurs nfc<br  ',
 86003: ' -  plancher chauffant',
 87160: ' - generation n 1 emission n 1',
 90590: 'climatisation reversible  - type split<br  ',
 94267: 'installation de chauffage  5732 1\xa0m²)\xa0:\r ',
 98728: 'installation de chauffage  432 3\xa0m²)\xa0:\r ',
 149386: ' -  radiateur bt avec robinet therm.',
 160826: ' - split ou multi-split split ou multi-split',
 162783: ' - split ou multi-split split ou multi-split',
 166307: 'au gaz naturel',
 182122: 'radiateurs nfc<br  ',
 223590: ' - daikin altherma 8kw radiateur a eau bt 45   plancher chauffant a eau 35 ',
 223960: ' - plancher chauffant plancher chauffant',
 339726: ' - atlantic alfea extensa duo+ plancher chauffant pcbt',
 372277: 'installation de chauffage 

In [356]:
sel=td003_ch.loc[td003_ch.td001_dpe_id.isin(dpe_list)]

sel.loc[sel.descriptif.str.contains('urbain')].descriptif.tolist()

AttributeError: 'DataFrame' object has no attribute 'valeur_renseignee'

In [286]:
id_="187094"

In [287]:
td001_sys.query(f'td001_dpe_id=="{id_}"')

,td001_dpe_id,tr002_type_batiment_id,gen_ch_lib_infer_concat,gen_ecs_lib_infer_concat,gen_ch_lib_final,is_ch_solaire_data,is_ch_solaire_txt,is_ch_solaire,type_installation_ch
3241,187094,1,chaudiere gaz indetermine + poele ou insert bois,chaudiere gaz indetermine,chaudiere bois + poele ou insert bois,False,NaN,False,individuel


In [288]:
td005_ecs.loc[td005_ecs.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '3319419',
  'td001_dpe_id': '187094',
  'tr011_sous_categorie_fiche_technique_id': '17',
  'valeur_renseignee': 'systeme de production d ecs principal surface couverte = 85.0 m2 chaudiere basse temperature gaz  sans veilleuse  murale installe en 1970. les pieces desservies sont contigues. la production est hors volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 150 l',
  'est_efface': '0'}]

In [289]:
td003_ch.loc[td003_ch.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '2029729',
  'td001_dpe_id': '187094',
  'tr007_type_descriptif_id': '11',
  'descriptif': 'radiateur avec robinet thermostatique sur chaudiere basse temperature gaz poel a bois',
  'est_efface': '0'}]

In [290]:
td005_ch.loc[td005_ch.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '3319417',
  'td001_dpe_id': '187094',
  'tr011_sous_categorie_fiche_technique_id': '16',
  'valeur_renseignee': 'systeme de chauffage principal surface couverte = 85.0 m2 programmation centrale avec controle de temperature radiateur a eau chaude avec robinet thermostatique installe en 2010 sur chaudiere basse temperature gaz  avec loi d eau  sans veilleuse  murale installee en 2010. distribution hydraulique haute temperature individuelle  avec canalisations non isolees. ch. solaire : non  production : indiv.',
  'est_efface': '0'},
 {'id': '3319418',
  'td001_dpe_id': '187094',
  'tr011_sous_categorie_fiche_technique_id': '16',
  'valeur_renseignee': 'systeme de chauffage numero 2 en complement du systeme principale poele a bois installe en 2010 production : indiv.',
  'est_efface': '0'}]

In [280]:
td003_ecs.loc[td003_ecs.td001_dpe_id==id_].to_dict(orient='records')


[]

In [263]:
gen_ch_lib_ft.loc[gen_ch_lib_ft.td001_dpe_id==id_]

,id,label,category,td001_dpe_id


In [433]:
m.loc[m.id=='174001369']

,id,label


In [416]:
energie_ch_ft.loc[energie_ch_ft.td001_dpe_id==id_]

,id,label,td001_dpe_id
92363,174001369,Charbon,9425890


In [ ]:
m.loc[gen_ch_lib_ft.td001_dpe_id==id_]

In [ ]:
m

In [266]:
999852

999852

In [471]:
data_to_search = td005_ch


In [474]:
id_col='id'
val_col='valeur_renseignee'
search_dict=gen_ch_search_dict_flat

In [477]:
es_client = setup_es_client()
bulk(es_client, gendata(index_name,data_to_search[id_col],data_to_search[val_col]))
# res_serie = search_from_search_dict(es_client,search_dict,index_name=index_name)

# res_table=res_serie.to_frame('label')

# res_table.index.name = 'id'
# res_table=res_table.reset_index()

(129490, [])

In [476]:
res_table.loc[res_table.id=='174001369']

,id,label


In [489]:
s_list = list()
for label,char_list in  search_dict.items():

    search_instruction =generate_instruction_from_list(char_list)
    search_body = {
        "query": {
            "query_string": {
                "query": search_instruction,
                "default_field": "text_to_analyze"
            },

        },

    }
    #es_client.count(index=index_name, body=search_body)

    a_dict=es_client.search(index=index_name, body=search_body,size=500000)

    hits = a_dict['hits']['hits']

    s=pd.Series(index=[el['_source']['id'] for el in hits])
    
    s[:]=label
    s_list.append(s)
s_all=pd.concat(s_list)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [482]:
[el['_source'] for el in hits if  el['_source']['id']=='174001369']

[{'id': '174001369',
  'text_to_analyze': 'poele au charbon   insert installe apres 2001 avec label flamme verte  systeme individuel)<br  '}]

In [485]:
s[:]=label

In [486]:
s

146290980    poele ou insert bois
44997988     poele ou insert bois
55337697     poele ou insert bois
57200175     poele ou insert bois
57204622     poele ou insert bois
                     ...         
12842927     poele ou insert bois
18027967     poele ou insert bois
18100042     poele ou insert bois
13645260     poele ou insert bois
17235737     poele ou insert bois
Length: 1171, dtype: object

174001369    poele ou insert bois
dtype: object

In [488]:
[el for el in s_all el =='174001369']

[]

In [481]:
hits

[{'_index': 'dpe_extract_text',
  '_type': '_doc',
  '_id': '8vwkinYBjBMnNQstRzG6',
  '_score': 31.21572,
  '_source': {'id': '146290980',
   'text_to_analyze': ' - chaudiere bois atmospherique avant 1978 poele ou insert bois avant 2ooo ou sans label flamme verte'}},
 {'_index': 'dpe_extract_text',
  '_type': '_doc',
  '_id': 'x_skinYBjBMnNQstOQUw',
  '_score': 30.094772,
  '_source': {'id': '44997988',
   'text_to_analyze': 'installation de chauffage avec insert ou poele bois ou biomasse en appoint  84\xa0m²)\xa0:\r - chauffage 1 : convecteurs nfc  energie : electrique)  sans equipement d intermittence\r - appoint 1 : insert  energie : bois)  equipement recent  apres 2001)  label flamme verte'}},
 {'_index': 'dpe_extract_text',
  '_type': '_doc',
  '_id': 'v_skinYBjBMnNQstOinR',
  '_score': 30.094772,
  '_source': {'id': '55337697',
   'text_to_analyze': 'installation de chauffage avec insert ou poele bois ou biomasse en appoint  120\xa0m²)\xa0:\r - chauffage 1 : radiateurs nfc  energ

# debug search

In [92]:
toto="""
systeme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 
"""
toto="""
 - collectif : solaire avec appoint chaudiere a condensation gaz naturel"""

toto="""

'type d equipement : chaudiere bois classe 4 energie : bois type d installation : individuelle  sans solaire description : standard  anciennete 0 distribution : production dans le volume chauffe  pieces alimentees contigues  :  '
"""
toto ="""
'installation d ecs 34 9\xa0m²\r - ecs 1 : chauffe-eau standard  energie : electrique) avec accumulation horizontale 75\xa0l ; production en volume habitable  alimentant des pieces non contigues'
"""
toto ="""
poele au charbon   insert installe apres 2001 avec label flamme verte  systeme individuel)<br  
"""
#toto = td005_ch.loc[td005_ch.td001_dpe_id==id_].to_dict(orient='records')[1]['valeur_renseignee']

toto ='chaudiere elec.'

In [93]:
'collective', 'collectives', 'collectif', 'collectifs'

('collective', 'collectives', 'collectif', 'collectifs')

In [94]:
index_name = 'dpe_extract_text'

In [95]:
search_instruction = '((reseau de chaleur) OR (reseaux de chaleurs) OR (reseaux de chaleur))'
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['ecs thermodynamique electrique(pac ou ballon)'])
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['abscence ecs solaire'])



In [96]:
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['chaudiere bois'])

search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['production mixte indetermine'])
search_instruction = generate_instruction_from_list(gen_ch_search_dict_flat['poele ou insert bois'])

In [97]:
search_instruction = generate_instruction_from_list(energie_dict['gaz'])
search_instruction = generate_instruction_from_list(installation_dict['collectif'])

In [98]:
#search_instruction='((mixte) OR (combine) OR (chauffage AND ecs) OR (lie ) OR (combine ) OR (idem))'

In [294]:
toto = ' - chauffage bois + chaudiere gaz basse temperature chaffoteaux inoa ventouse 25  ff n poele a granules bois artense cleo  radiateurs a eau + horloge'

In [295]:
from generate_dpe_annexes_scripts.text_matching_dict import *

In [297]:
search_instruction = generate_instruction_from_list(gen_ch_search_dict['chaudiere']['chaudiere bois'])

In [298]:
search_instruction

'((chaud.) OR (chaud) OR (chaudi*) OR (chaufferie) OR (chauudiere) OR (condens*)) AND NOT ((poele) OR (insert) OR (cuisin*)) AND ((bois) OR (bois//biomasse) OR (biomasse) OR (flamme AND verte) OR (granule*) OR (pellet*) OR (buche*))'

In [299]:
import time

In [300]:
search_body = {
    "query": {
        "query_string": {
            "query": search_instruction,
            "default_field": "text_to_analyze"
        },

    },

}

es_client = setup_es_client(index_name=index_name)
bulk(es_client, gendata(index_name,['id'],[toto]))

time.sleep(1)
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [219]:
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.1507283,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': 'fFpc4XYBDdqbXV0tC9dd',
    '_score': 1.1507283,
    '_source': {'id': 'id', 'text_to_analyze': 'bois/biomasse'}}]}}

In [60]:
bulk(es_client, gendata(index_name,['id'],[toto]))


(1, [])

In [53]:
es_client

<Elasticsearch([{}])>

In [54]:
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.8630463,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': 'MeKchHYBLPwkNvcNz20A',
    '_score': 0.8630463,
    '_source': {'id': 'id',
     'text_to_analyze': '\nsysteme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 \n'}}]}}

In [ ]:

s_all = search_from_search_dict(es_client,search_dict,index_name=index_name)

In [46]:
m_ch = m

In [81]:
id_=m_ch.vr_source_ft.str.contains('air')&m_ch.vr_source_ft.str.contains('eau')&m_ch.vr_source_ft.str.contains('pac')

In [76]:
m_ch.loc[m_ch.gen_ch_lib_ft.fillna('na').str.contains('air/eau')]

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm


In [77]:
m_ch.gen_ch_lib_<Q<

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm
0,1000015,convecteurs electriques nfc,convecteur electrique nfc :\r - type de produ...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteur electrique nfc\r,indetermine,indetermine,convecteurs electriques nfc,Electricité non renouvelable,Chauffage Individuel,3CL 2012
1,1000040,convecteurs electriques nfc,convecteurs electriques nfc systeme individue...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteurs electriques nfc systeme individuel),Chauffage Individuel,Chauffage Individuel,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
2,1000043,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1991,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
3,1000076,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur<br ...,indetermine,indetermine,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur,indetermine,indetermine,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
4,1000112,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145974,9402027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145975,9410001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145976,9421242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3CL 2012
145977,9421904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE


In [53]:
s=m_ch.loc[m_ch.nom_methode_dpe_norm=='FACTURE'].gen_ch_lib_ft.fillna('indetermine').value_counts()

In [57]:
s.loc['indetermine']/s.sum()

0.15338467987006182

In [55]:
(s=='indetermine').mean()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0.0

In [35]:
m_ch.groupby('nom_methode_dpe_norm').gen_ch_lib_ft.value_counts()

nom_methode_dpe_norm  gen_ch_lib_ft                                                              
3CL 2005              convecteurs electriques nfc                                                    45
                      radiateurs electriques                                                         15
                      panneaux rayonnants electriques nfc                                            13
                      chaudiere gaz basse temperature                                                12
                      chaudiere gaz standard                                                          9
                                                                                                     ..
FACTURE               chaudiere energie indetermine indetermine + convecteurs electriques nfc         1
                      chaudiere gaz basse temperature + chaudiere gaz indetermine                     1
                      chaudiere gaz indetermine + indetermine         

In [ ]:
m.type_installation_ch_ft.value_counts()

In [ ]:
m.loc[m.gen_ch_lib_ft=='indetermine'].head(470).to_dict(orient='records')